# Tested for `networks_2022`

In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork

In [4]:
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'

# Compare Torch net to Tensorflow net

In [5]:
# Torch network outputs
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

y_hats = []

with h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r') as f_in:
    for ind in range(len(f_in['data'])):
        word, genre = net(torch.tensor(np.reshape( f_in['data'][ind],(164,400))))
        
        y_hat = int(torch.argmax(word))
        y_hats.append(y_hat)
        if len(y_hats) > 20:
            break

print(y_hats[:20])

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


[229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46]


In [6]:
# Tensorflow network outputs
tf_y_hats = np.array([229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259])
tf_y_hats_no_Dropout = [229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259]
print(tf_y_hats[:20])

[229 312 525   3 393   5   6   7   8   9  10  11  12  13 126 108  15 298
  17  46]


In [7]:
# Number of matches
print(len(np.where(tf_y_hats == np.array(y_hats))[0]))

0


/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [14]:
correct = []
for ii in range(len(tf_y_hats)):
    if tf_y_hats[ii] == labels[ii]:
        correct.append(1)
    else:
        correct.append(0)

In [15]:
print(np.mean(correct))

0.47


# Compare Torch net to Ground truth labels
Also save the info on which examples the net gets correct/incorrect

In [10]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [11]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [12]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [ ]:
y_hats = []
corrects = []

with h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r') as f_in:
    for ind in range(len(f_in['data'])):
        word, genre =net(torch.tensor(np.reshape( f_in['data'][ind],(164,400))))
        
        y_hat = int(torch.argmax(word))
        y_hats.append(y_hat)
        corrects.append(y_hat == labels[ind])

In [ ]:
# Save net predictions and mistakes to file
with open(f"{engram_dir}PsychophysicsWord2017W_net_performance.p", 'wb') as f:
    results = {
        'net_pred': np.array(y_hats),
        'net_mistakes': np.logical_not(corrects)
    }
    pickle.dump(results, f)

# Compare Predified-Net with Torch Net
The outputs should match at timestep 0 of the predified-net

In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'

In [ ]:
# Specify which predified network you want to test
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'
p_layers = 'All Layers'

In [ ]:
# Torch network outputs from before
print(y_hats[:20])

In [ ]:
# Predified network outputs
pnet = PNetClass(net, build_graph=True)
pnet.load_state_dict(torch.load(
    f"{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth",
    map_location='cpu'
    ))
pnet.to(DEVICE)
pnet.build_graph = False        # only required for training
pnet.eval()

y_hats = []

f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')
for ind in range(len(f_in['data'])):
    pnet.reset()
    _input = torch.tensor(np.reshape(f_in['data'][ind],(164,400))).to(DEVICE)
    word, genre = pnet(_input)
    y_hat = int(torch.argmax(word))
    y_hats.append(y_hat)
    if len(y_hats) > 20:
        break
        
print(y_hats[:20])